## Tutorial: 

#### Configure the Environment

Import the chi example API calls, set the project name and region, and set various names and attributes to use in the tutorial. 

In [ ]:
import json
import os
import chi

from chi.server import *
from chi.lease import *
from chi.network import *

from datetime import datetime, timedelta
from dateutil import tz

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'
VERBOSE=False  #Set to true for extra output

#Config with your project and site
chi.use_site('CHI@UC')
chi.set('project_name', 'CH-XXXXXX')

#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

#GENI Pem file
geni_pem_file='/my_exogini_pem.pem'

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")+'12'
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server Config
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type=""
server_count=1

#Network Config
physical_network='exogeni'
cidr='192.168.42.0/24'

uc_allocation_start='192.168.42.101'
uc_allocation_end='192.168.42.150'
uc_gateway='192.168.42.100'
tacc_allocation_start='192.168.42.201'
tacc_allocation_end='192.168.42.250'
tacc_gateway='192.168.42.200'

## Create Chicago Network and Server

#### Create a Lease at Chicago

In [ ]:
#Set the region
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

In [ ]:
#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(uc_lease, indent=2))
else:
    print('uc_lease: ' + uc_lease['name'] + ', ' + uc_lease['id'])

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
uc_lease = get_lease(get_lease_id(lease_name))

uc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', uc_lease['reservations']))[0]['id']
uc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', uc_lease['reservations']))[0]['id']
uc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', uc_lease['reservations']))[0]['id']

print("uc_compute_reservation_id: " + uc_compute_reservation_id)
print("uc_network_reservation_id: " + uc_network_reservation_id)
print("uc_floatingip_reservation_id: " + uc_floatingip_reservation_id)

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
chi.use_site('CHI@UC')    # Optional, defaults to 'CHI@UC'

#Get the network
uc_network = get_network(get_network_id(network_name))
uc_network_id = uc_network['id']
uc_network_vlan = uc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(uc_network, indent=2))
else:
    print('uc_network: ' + uc_network['name'] + ', ' + uc_network['id'])
    print('uc segmenation ID/VLAN: ' + str(uc_network_vlan))

#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = create_subnet(subnet_name, 
                       uc_network_id, 
                       cidr=cidr,
                      allocation_pool_start=uc_allocation_start,
                      allocation_pool_end=uc_allocation_end,
                      gateway_ip=uc_gateway)



#print(json.dumps(subnet, indent=2))

In [ ]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

## Create TACC Network and Server

#### Create a Lease at Chicago

In [ ]:
node_type="compute_cascadelake"

#Set the region
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

In [ ]:
chi.use_site('CHI@TACC')

#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))
    
#Print the lease info
if VERBOSE:
    print(json.dumps(tacc_lease, indent=2))
else:
    print('tacc_lease: ' + tacc_lease['name'] + ', ' + tacc_lease['id'])

#### Get the Reservations

Each lease contains one or more reservations. The individual reservation IDs are required to instantiate resources. You can [get the lease](../modules-python/reservations/get_lease_by_name.ipynb) and separate the reservation IDs for compute, network, and floating IPs using the technique below.

In [ ]:
#Get the lease by name
tacc_lease = get_lease(get_lease_id(lease_name))

tacc_compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', tacc_lease['reservations']))[0]['id']
tacc_network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', tacc_lease['reservations']))[0]['id']
tacc_floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', tacc_lease['reservations']))[0]['id']

print("tacc_compute_reservation_id: " + tacc_compute_reservation_id)
print("tacc_network_reservation_id: " + tacc_network_reservation_id)
print("tacc_floatingip_reservation_id: " + tacc_floatingip_reservation_id)

#### Get the Network

Getting the network is not required for the remainder of the tutorial. However, it is a good test to see if your network reservation has become active. The [get_network_by_name](../modules-python/network/get_network_by_name.ipynb) call will fail if a network with that name does not yet exits. It will also fail if a network with the same name already exists (likely from a previous run of this notebook).

In [ ]:
chi.use_site('CHI@TACC')    # Optional, defaults to 'CHI@UC'

#Get the network
tacc_network = get_network(get_network_id(network_name))
tacc_network_id = tacc_network['id']
tacc_network_vlan = tacc_network['provider:segmentation_id']

if VERBOSE:
    print(json.dumps(tacc_network, indent=2))
else:
    print('tacc_network: ' + tacc_network['name'] + ', ' + tacc_network['id'])
    print('tacc segmenation ID/VLAN: ' + str(tacc_network_vlan))

#### Add a subnet

[Adds a subnet](../modules-python/network/add_subnet.ipynb) to the reserved network. 

In [ ]:
subnet = create_subnet(subnet_name, 
                       tacc_network_id, 
                       cidr=cidr, 
                      allocation_pool_start=tacc_allocation_start,
                      allocation_pool_end=tacc_allocation_end,
                      gateway_ip=tacc_gateway)

 

#print(json.dumps(subnet, indent=2))

In [ ]:
if VERBOSE:
    print(json.dumps(subnet, indent=2))
else:
    print('Subnet: ' + subnet['name'] + ', ' + subnet['id'])

#### Add a Router

TODO: add links here

In [ ]:
router = create_router(router_name, gw_network_name='public')
#print(json.dumps(router, indent=2))

if VERBOSE:
    print(json.dumps(router, indent=2))
else:
    print('Router: ' + router['name'] + ', ' + router['id'])

#### Attach the Router and Subnet

TODO: Add links here

In [ ]:
add_subnet_to_router(get_router_id(router_name), get_subnet_id(subnet_name))

## Start the Servers

#### Start the UC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.use_site('CHI@UC')  
#create the server
server = create_server(server_name, 
                       reservation_id=uc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.use_site('CHI@UC')
server_id = get_server_id(server_name)
floating_ip = associate_floating_ip(server_id)

print('Floating IP: ' + str(floating_ip))

#### Start the TACC Server

Use the compute_reservation_id to [create the server](../modules-python/servers/create_server.ipynb).

In [ ]:
chi.use_site('CHI@TACC')

print("server_name " + server_name)
print("tacc_compute_reservation_id " + tacc_compute_reservation_id)
print("key_name " + key_name) 
print("network_name " + network_name)
print("image_name " + image_name)
print("flavor_name " + flavor_name)

#create the server
server = create_server(server_name, 
                       reservation_id=tacc_compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name, 
                       image_name=image_name, 
                       flavor_name=flavor_name)


#### Associate the Floating IP   
TODO: need to find floating_ip from the reservation that was just made

In [ ]:
chi.use_site('CHI@TACC')  

server_id = get_server_id(server_name)
floating_ip = associate_floating_ip(server_id)

print('Floating IP: ' + str(floating_ip))

## Stitch the Circuit using ExoGENI

Note: The ExoGENI
steps require a valid GENI certificate at the path specified and a public/private keypair in ~/.ssh (run ssh-keygen with default inputs)

#### Create the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch create -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

#### Check the Status of the Circuit

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch status -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

## Clean Up Resources

### Delete Stitched Circuit using ExoGENI

In [ ]:
%%script env uc_vlan="$uc_network_vlan" tacc_vlan="$tacc_network_vlan" geni_pem="$geni_pem_file" bash

echo 'uc_vlan ' $uc_vlan ', tacc_vlan ' $tacc_vlan ', geni_pem ' $geni_pem
xoStitch delete -sp1 uc -vlan1 $uc_vlan -sp2 tacc -vlan2 $tacc_vlan -c $geni_pem

### Delete TACC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('CHI@TACC')

In [ ]:
delete_server(get_server_id(server_name))

#### De-configure Network
TODO: break up into steps

In [ ]:
remove_subnet_from_router(router['id'], subnet['id'])

In [ ]:
delete_router(router['id'])

In [ ]:
delete_subnet(subnet['id'])

In [ ]:
delete_network(tacc_network['id'])

#### Release Lease

In [ ]:
delete_lease(tacc_lease['id'])

### Delete UC Resources

[Delete the server](../modules-python/servers/delete_server.ipynb) using its name.

In [ ]:
chi.use_site('CHI@UC')

In [ ]:
delete_server(get_server_id(server_name))

#### De-configure Network
TODO: break up into steps

In [ ]:
remove_subnet_from_router(get_router_id(router_name), get_subnet_id(subnet_name))

In [ ]:
delete_router(get_router_id(router_name))

In [ ]:
delete_subnet(get_subnet_id(subnet_name))

In [ ]:
delete_network(uc_network['id'])

#### Release Lease

In [ ]:
delete_lease(uc_lease['id'])